# Quickmaths PyRVA
June 23, 2022

## Challenge
You must solve 1000 of these math problems that are outputted in the following format {number} {operation} {number} to get the flag. 
Division is integer division using the // operator. 
The input is being checked through python input() function. 
Good luck! 

## Connect
[Netcat](https://www.mankier.com/1/netcat) is a command line utility that for creating network socket connections.

Every machine on the network is connectable by it's ip address (routing) through ports (application) based on a server/client principle.

Your machine (client) can make a request to a port of a remote machine (server) and if there is an application listening for connections on that port, a socket connection can be established.

*Internet Assigned Numbers Authority* (IANA) maintains a [list](https://www.iana.org/assignments/service-names-port-numbers/service-names-port-numbers.txt) of "well-known" ports, but truthfully, these port assignments are arbitrary and can be re-used whenever the client and server are both in agreement. Generally speaking, you won't know if there is an application listening on a machine at a given port until you try to connect to it. If you get a response, we can assume the port is open and ready for connections.

Today's challenge is being hosted at this address and port:

In [233]:
# nc 44.202.214.112 1228

By using a tool like [American Registry for Internet Numbers (ARIN)](https://search.arin.net/rdap/?query=44.202.214.112), we can discover that this address is owned by Amazon and the prompt tells us it is serving a python script at the available port.


It's going to take you a while to solve that many math problems manually or it might be easier to find a way to automate the process...

## Python solution

#### Create a virtual environment in your working directory.

```
python -m venv venv
source venv/bin/activate
```

#### Import

Feel free to experiment with any set of tools you would like, but one such tool that creates a wrapper for rapid development with socket connections is [PwnTools](https://github.com/Gallopsled/pwntools/blob/dev/examples/remote.py).

In [234]:
from pwn import *

#### Variables

In [235]:
remote_host = "44.202.214.112"
remote_port = 1228

#### Create a remote connection

In [236]:
conn = remote(remote_host, remote_port)

[x] Opening connection to 44.202.214.112 on port 1228
[x] Opening connection to 44.202.214.112 on port 1228: Trying 44.202.214.112
[+] Opening connection to 44.202.214.112 on port 1228: Done


#### Read from remote socket

In [237]:
res = b''
while res != b'\n':
    res = conn.recvline()
    print(res.decode())

You must solve 1000 of these math problems that are outputted in the following format {number} {operation} {number} to get the flag. 

Division is integer division using the // operator. 

The input is being checked through python input() function. 

Good luck! 





#### Write to remote socket

In [238]:
# conn.send(b'1\n')

#### A function to parse the next line

In [239]:
def parse_next_line(conn):
    raw_problem = conn.readline()
    problem = raw_problem.decode().split(' ')
    term1 = int(problem[0])
    operand = problem[1]
    term2 = int(problem[2].replace('\n', ''))
    return term1, operand, term2

#### A function to perform calculations

In [240]:
def do_maths(term1: int, operand: str, term2: int) -> int:
    match operand:
        case '//':
            if term2 == 0:
                raise ZeroDivisionError("I cannot divide by zero")
            return term1 // term2
        case '*':
            return term1 * term2
        case '-':
            return term1 - term2
        case '+':
            return term1 + term2
        case _:
            raise RuntimeError("Unexpected operand %s received" % operand)

#### A function to automate the responses

In [241]:
def send_response(result: str) -> None:
    conn.sendline(result.encode())

In [242]:
count = 0
how_i_am_doing = {'correct': 0, 'incorrect': 0}
while count < 1000:
    term1, operand, term2 = parse_next_line(conn)
    result = str(do_maths(term1, operand, term2))
    # print(result)
    send_response(result)
    response = conn.recvline().decode().replace('\n', '').lower()
    print(response)
    if response == "correct!":
        how_i_am_doing['correct'] += 1
    else:
        how_i_am_doing['incorrect'] += 1
    count += 1

print(conn.recvline().decode().replace('\n', ''))
print(how_i_am_doing)

conn.close()

correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
correct!
c

#### Try it yourself...
This challenge was originally seen during the [2022 UMASS CTF](https://ctftime.org/event/1561/) event. You can find the original source code here:

https://github.com/UMassCybersecurity/UMassCTF-2022-challenges/tree/main/misc/quickmaths/src



##### Run locally with Docker

[Docker](https://www.docker.com/products/docker-desktop/) is virtualization software for running apps in a sandboxed environment on your host machine.

After installing the docker daemon, run these commands from the folder containing the `Dockerfile`.

```
docker build -t quickmaths .
docker run --name quickmaths --rm -d -p 1228:1228 quickmaths
```
`-t` will create a new container image with the given name.

`--name` can be used to give the running container a name instead of a randomly generated name.

`--rm` will delete the container when it is stopped.

`-d` will start the container as a background process.

`-p` will map the container's port to your localhost.

##### Stop the container and delete the image
```
docker stop quickmaths
docker rmi quickmaths
```
